In [1]:
import os

# Check if we're in the root directory of rl-llm repo
if not os.path.basename(os.getcwd()) == 'rl-llm':
    # If we're in a subdirectory of rl-llm, find the root and cd to it
    current_path = os.getcwd()
    while os.path.basename(current_path) != 'rl-llm' and os.path.dirname(current_path) != current_path:
        current_path = os.path.dirname(current_path)
    
    if os.path.basename(current_path) == 'rl-llm':
        os.chdir(current_path)
        print(f"Changed directory to: {current_path}")
    else:
        print("Not in rl-llm repository structure")
else:
    print("Already in rl-llm root directory")

print(f"Current directory: {os.getcwd()}")

Changed directory to: /home/ubuntu/rl-llm
Current directory: /home/ubuntu/rl-llm


In [2]:
from src import EnvManager, sample_episodes
import os
import torch
from transformers import AutoTokenizer
from trl import AutoModelForCausalLMWithValueHead
from peft import PeftConfig

/home/ubuntu/rl-llm/.venv/lib/python3.10/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment BabyAI-GoTo-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/ubuntu/rl-llm/.venv/lib/python3.10/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment BabyAI-Open-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/ubuntu/rl-llm/.venv/lib/python3.10/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment BabyAI-Pickup-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/ubuntu/rl-llm/.venv/lib/python3.10/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment BabyAI-PutNext-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
2025-04-12 21:19:22.514280: E external/

In [3]:
def load_model_with_peft_and_vhead(model_id, revision=None):
    """
    Load a model with both PEFT adapters and value head
    
    Args:
        model_id: Path to model or HF hub model ID
        device: Device to load the model to
        revision: Specific model revision/commit hash to load
    
    Returns:
        model: The loaded model with adapters and value head
        tokenizer: The associated tokenizer
    """
    
    # Get PEFT config to find base model
    peft_config = PeftConfig.from_pretrained(model_id, revision=revision)
    base_model_id = peft_config.base_model_name_or_path
    
    print(f"Base model: {base_model_id}")
    print(f"Loading with PEFT adapters from: {model_id}")
    if revision:
        print(f"Using revision: {revision}")

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(base_model_id, padding_side='left')
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    # Initialize model with value head from the base model
    model = AutoModelForCausalLMWithValueHead.from_pretrained(
        base_model_id,
        device_map="auto",
    )
    
    # Now load the PEFT adapter weights
    model = model.from_pretrained(model_id, device_map="auto", revision=revision)
    
    return model, tokenizer


def load_base_model_with_vhead(model_id):
    """
    Load a base model with value head
    
    Args:
        model_id: Path to model or HF hub model ID
    
    Returns:
        model: The loaded model with value head
        tokenizer: The associated tokenizer
    """
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side='left')
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    print(f"Loading base model with value head from: {model_id}")
    
    # Initialize model with value head
    model = AutoModelForCausalLMWithValueHead.from_pretrained(
        model_id,
        device_map="auto",
    )
    
    return model, tokenizer


# Example usage
model_id = "CatkinChen/final_runs-Reasoning_trial_2_dist_3"
revision = None # "cf75726c11f5b5867107f56efc8f55439635f1c6" 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, tokenizer = load_model_with_peft_and_vhead(model_id, revision)
# model, tokenizer = load_base_model_with_vhead("meta-llama/Llama-3.2-3B-Instruct")

# Test the model with a simple query
prompt = "You are in a room with a key. The instruction is to: pick up the key"
inputs = tokenizer(prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=50, do_sample=True, temperature=0.7)
print("\nModel output:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Base model: meta-llama/Llama-3.2-3B-Instruct
Loading with PEFT adapters from: CatkinChen/final_runs-Reasoning_trial_2_dist_3


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Model output:
You are in a room with a key. The instruction is to: pick up the key and put it on the table. You then need to go to the door and open the door with the key. 

## Step 1: Pick up the key
First, we need to pick up the key from the room.

## Step 


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

generation_kwargs = {
    "max_new_tokens": 20,
    "do_sample": True,
    "top_k": 50,
    "top_p": 0.9,
    "temperature": 0.7,
    "pad_token_id": tokenizer.pad_token_id,
}

In [5]:
env_id = "BabyAI-Pickup-v0"
reasoning_flag = True
env_ids = [env_id]
num_envs = 6
env_managers = [
    EnvManager(
        env_ids, 
        invalid_action_penalty=-2,
        consecutive_invalid_actions_allowed=5,
        reasoning_flag=reasoning_flag,
        num_dists=3,
    )
    for i in range(num_envs)
]

stats, contexts = sample_episodes(
    envs=env_managers,
    tokenizer=tokenizer,
    model=model,
    generation_kwargs=generation_kwargs,
    device=device,
    number_of_episodes=50,
    context_window=5,
    reasoning_flag=reasoning_flag,
)
success_rate = sum(stats["success"]) / len(stats["success"])
avg_reward = sum(stats["rewards"]) / len(stats["rewards"])
std_reward = torch.std(torch.tensor(stats["rewards"], dtype=torch.float32)).item()
avg_episode_length = sum(stats["episode_lengths"]) / len(stats["episode_lengths"])
std_episode_length = torch.std(torch.tensor(stats["episode_lengths"], dtype=torch.float32)).item()
avg_num_invalid_actions = sum(stats["num_invalid_actions"]) / len(stats["num_invalid_actions"])

print(f"Summary: {env_id} with {'Reasoning' if reasoning_flag else 'No Reasoning'}")
print(f"Sample size: {len(stats['success'])}")
print(f"Success Rate: {success_rate:.2f}")
print(f"Average Reward: {avg_reward:.2f} ± {std_reward:.2f}")
print(f"Average Episode Length: {avg_episode_length:.2f} ± {std_episode_length:.2f}")
print(f"Average Number of Invalid Actions: {avg_num_invalid_actions:.2f}")
print()

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 
Summary: BabyAI-Pickup-v0 with Reasoning
Sample size: 50
Success Rate: 0.92
Average Reward: 0.81 ± 0.26
Average Episode Length: 14.08 ± 18.41
Average Number of Invalid Actions: 1.04



In [6]:
env_id = "BabyAI-PickupTest-v0"
reasoning_flag = True
env_ids = [env_id]
num_envs = 6
env_managers = [
    EnvManager(
        env_ids, 
        invalid_action_penalty=-2,
        consecutive_invalid_actions_allowed=5,
        reasoning_flag=reasoning_flag,
        num_dists=3,
    )
    for i in range(num_envs)
]

stats, contexts = sample_episodes(
    envs=env_managers,
    tokenizer=tokenizer,
    model=model,
    generation_kwargs=generation_kwargs,
    device=device,
    number_of_episodes=50,
    context_window=5,
    reasoning_flag=reasoning_flag,
)
success_rate = sum(stats["success"]) / len(stats["success"])
avg_reward = sum(stats["rewards"]) / len(stats["rewards"])
std_reward = torch.std(torch.tensor(stats["rewards"], dtype=torch.float32)).item()
avg_episode_length = sum(stats["episode_lengths"]) / len(stats["episode_lengths"])
std_episode_length = torch.std(torch.tensor(stats["episode_lengths"], dtype=torch.float32)).item()
avg_num_invalid_actions = sum(stats["num_invalid_actions"]) / len(stats["num_invalid_actions"])

print(f"Summary: {env_id} with {'Reasoning' if reasoning_flag else 'No Reasoning'}")
print(f"Sample size: {len(stats['success'])}")
print(f"Success Rate: {success_rate:.2f}")
print(f"Average Reward: {avg_reward:.2f} ± {std_reward:.2f}")
print(f"Average Episode Length: {avg_episode_length:.2f} ± {std_episode_length:.2f}")
print(f"Average Number of Invalid Actions: {avg_num_invalid_actions:.2f}")
print()

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 
Summary: BabyAI-PickupTest-v0 with Reasoning
Sample size: 50
Success Rate: 0.86
Average Reward: 0.76 ± 0.32
Average Episode Length: 15.74 ± 20.92
Average Number of Invalid Actions: 1.20



In [7]:
model_id = "CatkinChen/final_runs-No_Reasoning_trial_2_dist_3"
revision = None # "cf75726c11f5b5867107f56efc8f55439635f1c6" 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, tokenizer = load_model_with_peft_and_vhead(model_id, revision)

Base model: meta-llama/Llama-3.2-3B-Instruct
Loading with PEFT adapters from: CatkinChen/final_runs-No_Reasoning_trial_2_dist_3


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
model, tokenizer = load_base_model_with_vhead("meta-llama/Llama-3.2-3B-Instruct")

env_id = "BabyAI-Pickup-v0"
reasoning_flag = True
env_ids = [env_id]
num_envs = 6
env_managers = [
    EnvManager(
        env_ids, 
        invalid_action_penalty=-2,
        consecutive_invalid_actions_allowed=5,
        reasoning_flag=reasoning_flag,
        num_dists=3,
    )
    for i in range(num_envs)
]

stats, contexts = sample_episodes(
    envs=env_managers,
    tokenizer=tokenizer,
    model=model,
    generation_kwargs=generation_kwargs,
    device=device,
    number_of_episodes=50,
    context_window=5,
    reasoning_flag=reasoning_flag,
)
success_rate = sum(stats["success"]) / len(stats["success"])
avg_reward = sum(stats["rewards"]) / len(stats["rewards"])
std_reward = torch.std(torch.tensor(stats["rewards"], dtype=torch.float32)).item()
avg_episode_length = sum(stats["episode_lengths"]) / len(stats["episode_lengths"])
std_episode_length = torch.std(torch.tensor(stats["episode_lengths"], dtype=torch.float32)).item()
avg_num_invalid_actions = sum(stats["num_invalid_actions"]) / len(stats["num_invalid_actions"])

print(f"Summary: {env_id} with {'Reasoning' if reasoning_flag else 'No Reasoning'}")
print(f"Sample size: {len(stats['success'])}")
print(f"Success Rate: {success_rate:.2f}")
print(f"Average Reward: {avg_reward:.2f} ± {std_reward:.2f}")
print(f"Average Episode Length: {avg_episode_length:.2f} ± {std_episode_length:.2f}")
print(f"Average Number of Invalid Actions: {avg_num_invalid_actions:.2f}")
print()

Loading base model with value head from: meta-llama/Llama-3.2-3B-Instruct


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 
Summary: BabyAI-Pickup-v0 with Reasoning
Sample size: 50
Success Rate: 0.16
Average Reward: 0.10 ± 0.25
Average Episode Length: 28.28 ± 26.17
Average Number of Invalid Actions: 4.80



In [12]:
model, tokenizer = load_base_model_with_vhead("meta-llama/Llama-3.2-3B-Instruct")

env_id = "BabyAI-Pickup-v0"
reasoning_flag = False
env_ids = [env_id]
num_envs = 6
env_managers = [
    EnvManager(
        env_ids, 
        invalid_action_penalty=-2,
        consecutive_invalid_actions_allowed=5,
        reasoning_flag=reasoning_flag,
        num_dists=3,
    )
    for i in range(num_envs)
]

stats, contexts = sample_episodes(
    envs=env_managers,
    tokenizer=tokenizer,
    model=model,
    generation_kwargs=generation_kwargs,
    device=device,
    number_of_episodes=50,
    context_window=5,
    reasoning_flag=reasoning_flag,
)
success_rate = sum(stats["success"]) / len(stats["success"])
avg_reward = sum(stats["rewards"]) / len(stats["rewards"])
std_reward = torch.std(torch.tensor(stats["rewards"], dtype=torch.float32)).item()
avg_episode_length = sum(stats["episode_lengths"]) / len(stats["episode_lengths"])
std_episode_length = torch.std(torch.tensor(stats["episode_lengths"], dtype=torch.float32)).item()
avg_num_invalid_actions = sum(stats["num_invalid_actions"]) / len(stats["num_invalid_actions"])

print(f"Summary: {env_id} with {'Reasoning' if reasoning_flag else 'No Reasoning'}")
print(f"Sample size: {len(stats['success'])}")
print(f"Success Rate: {success_rate:.2f}")
print(f"Average Reward: {avg_reward:.2f} ± {std_reward:.2f}")
print(f"Average Episode Length: {avg_episode_length:.2f} ± {std_episode_length:.2f}")
print(f"Average Number of Invalid Actions: {avg_num_invalid_actions:.2f}")
print()

Loading base model with value head from: meta-llama/Llama-3.2-3B-Instruct


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 
Summary: BabyAI-Pickup-v0 with No Reasoning
Sample size: 50
Success Rate: 0.38
Average Reward: 0.24 ± 0.33
Average Episode Length: 52.64 ± 22.58
Average Number of Invalid Actions: 3.82



In [8]:
env_id = "BabyAI-Pickup-v0"
reasoning_flag = False
env_ids = [env_id]
num_envs = 6
env_managers = [
    EnvManager(
        env_ids, 
        invalid_action_penalty=-2,
        consecutive_invalid_actions_allowed=5,
        reasoning_flag=reasoning_flag,
        num_dists=3,
    )
    for i in range(num_envs)
]

stats, contexts = sample_episodes(
    envs=env_managers,
    tokenizer=tokenizer,
    model=model,
    generation_kwargs=generation_kwargs,
    device=device,
    number_of_episodes=50,
    context_window=5,
    reasoning_flag=reasoning_flag,
)
success_rate = sum(stats["success"]) / len(stats["success"])
avg_reward = sum(stats["rewards"]) / len(stats["rewards"])
std_reward = torch.std(torch.tensor(stats["rewards"], dtype=torch.float32)).item()
avg_episode_length = sum(stats["episode_lengths"]) / len(stats["episode_lengths"])
std_episode_length = torch.std(torch.tensor(stats["episode_lengths"], dtype=torch.float32)).item()
avg_num_invalid_actions = sum(stats["num_invalid_actions"]) / len(stats["num_invalid_actions"])

print(f"Summary: {env_id} with {'Reasoning' if reasoning_flag else 'No Reasoning'}")
print(f"Sample size: {len(stats['success'])}")
print(f"Success Rate: {success_rate:.2f}")
print(f"Average Reward: {avg_reward:.2f} ± {std_reward:.2f}")
print(f"Average Episode Length: {avg_episode_length:.2f} ± {std_episode_length:.2f}")
print(f"Average Number of Invalid Actions: {avg_num_invalid_actions:.2f}")
print()

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 
Summary: BabyAI-Pickup-v0 with No Reasoning
Sample size: 50
Success Rate: 0.86
Average Reward: 0.74 ± 0.33
Average Episode Length: 17.80 ± 21.03
Average Number of Invalid Actions: 0.04



In [9]:
env_id = "BabyAI-PickupTest-v0"
reasoning_flag = False
env_ids = [env_id]
num_envs = 6
env_managers = [
    EnvManager(
        env_ids, 
        invalid_action_penalty=-2,
        consecutive_invalid_actions_allowed=5,
        reasoning_flag=reasoning_flag,
        num_dists=3,
    )
    for i in range(num_envs)
]

stats, contexts = sample_episodes(
    envs=env_managers,
    tokenizer=tokenizer,
    model=model,
    generation_kwargs=generation_kwargs,
    device=device,
    number_of_episodes=50,
    context_window=5,
    reasoning_flag=reasoning_flag,
)
success_rate = sum(stats["success"]) / len(stats["success"])
avg_reward = sum(stats["rewards"]) / len(stats["rewards"])
std_reward = torch.std(torch.tensor(stats["rewards"], dtype=torch.float32)).item()
avg_episode_length = sum(stats["episode_lengths"]) / len(stats["episode_lengths"])
std_episode_length = torch.std(torch.tensor(stats["episode_lengths"], dtype=torch.float32)).item()
avg_num_invalid_actions = sum(stats["num_invalid_actions"]) / len(stats["num_invalid_actions"])

print(f"Summary: {env_id} with {'Reasoning' if reasoning_flag else 'No Reasoning'}")
print(f"Sample size: {len(stats['success'])}")
print(f"Success Rate: {success_rate:.2f}")
print(f"Average Reward: {avg_reward:.2f} ± {std_reward:.2f}")
print(f"Average Episode Length: {avg_episode_length:.2f} ± {std_episode_length:.2f}")
print(f"Average Number of Invalid Actions: {avg_num_invalid_actions:.2f}")
print()

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 
Summary: BabyAI-PickupTest-v0 with No Reasoning
Sample size: 50
Success Rate: 0.80
Average Reward: 0.70 ± 0.36
Average Episode Length: 19.98 ± 23.05
Average Number of Invalid Actions: 0.04



In [ ]:
# Reasoning Train: Average Reward: 0.81 ± 0.26
# Reasoning Test: Average Reward: 0.76 ± 0.32
# Non-Reasoning Train: Average Reward: 0.74 ± 0.33
# Non-Reasoning Test: Average Reward: 0.70 ± 0.36

In [10]:
# Summary: BabyAI-Pickup-v0 with Reasoning (base model)
# Sample size: 10
# Success Rate: 0.50
# Average Reward: 0.37 ± 0.40
# Average Episode Length: 14.00 ± 9.76
# Average Number of Invalid Actions: 3.60

# Summary: BabyAI-Pickup-v0 with Reasoning (finetuned model but not run until convergence)
# Sample size: 10
# Success Rate: 0.70
# Average Reward: 0.56 ± 0.41
# Average Episode Length: 15.10 ± 10.79
# Average Number of Invalid Actions: 2.50

# Summary: BabyAI-Pickup-v0 with Reasoning (finetuned for longer)
# Sample size: 12
# Success Rate: 0.92
# Average Reward: 0.80 ± 0.26
# Average Episode Length: 9.33 ± 4.29
# Average Number of Invalid Actions: 0.50